# Should we change the simulator to change the batch?

In [4]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer

# Define repressilator model
def model(variables, t, params):

    m1, p1, m2, p2, m3, p3 = variables
    k1, k2 = params #only 2 ks are parameters to infer
    k3 = 246.96291990024542
    a1 = a2 = a3 = 24.78485282457379
    g1 = g2 = g3 = 0.024884149937163258
    n1 = n2 = n3 = 5
    b1 = b2 = b3 = 33.82307682700831
    dm1 = dm2 = dm3 = 1.143402097500176
    dp1 = dp2 = dp3 = 0.7833664565550977

    dm1dt = -dm1 * m1 + (a1 / (1 + ((1/k1) * p2) ** n1)) + g1
    dp1dt = (b1 * m1) - (dp1 * p1)
    dm2dt = -dm2 * m2 + (a2 / (1 + ((1/k2) * p3) ** n2)) + g2
    dp2dt = (b2 * m2) - (dp2 * p2)
    dm3dt = -dm3 * m3 + (a3 / (1 + ((1/k3) * p1) ** n3)) + g3
    dp3dt = (b3 * m3)-(dp3 * p3)
    
    return [dm1dt, dp1dt, dm2dt, dp2dt, dm3dt, dp3dt]

# Define the true parameters for k
true_params = np.array([
    246.96291990024542, 246.96291990024542,])

# Establish prior from 0-250
num_dim = 2
prior = utils.BoxUniform(low=10**-2 * torch.ones(num_dim), high=250 * torch.ones(num_dim))

In [3]:

import warnings
from typing import Any, Callable, Dict, Optional, Sequence, Tuple, Union, cast

import torch
from numpy import ndarray
from pyknos.nflows import flows
from scipy.stats._distn_infrastructure import rv_frozen
from scipy.stats._multivariate import multi_rv_frozen
from torch import Tensor, float32, nn
from torch.distributions import Distribution, Uniform

from sbi.types import Array
from sbi.utils.sbiutils import warn_on_iid_x, within_support
from sbi.utils.torchutils import BoxUniform, atleast_2d
from sbi.utils.user_input_checks_utils import (
    CustomPriorWrapper,
    MultipleIndependent,
    PytorchReturnTypeWrapper,
    ScipyPytorchWrapper,
)

def ensure_batched_simulator(simulator: Callable, prior) -> Callable:
    """Return a simulator with batched output.

    Return the unchanged simulator if it can already simulate multiple parameter
    vectors per call. Otherwise, wrap as simulator with batched output (leading batch
    dimension of shape [1]).
    """

    is_batched_simulator = True
    try:
        batch_size = 2
        # The simulator must return a matching batch dimension and data.
        output_shape = simulator(prior.sample((batch_size,))).shape
        assert len(output_shape) > 1
        assert output_shape[0] == batch_size
    except Exception:
        is_batched_simulator = False

    return simulator if is_batched_simulator else get_batch_loop_simulator(simulator)

6000

In [2]:
# We define the time variable (0-100 in 1000 steps), the initial conditions, etc.
num_timesteps = 1000
num_trajectories = 6
initial_conditions = np.array([0, 1, 0, 3, 0, 2])
t = np.linspace(0, 100, num_timesteps)

def simulator(parameter_set):
    initial_conditions = np.array([0, 1, 0, 3, 0, 2])
    t = np.linspace(0, 100, num_timesteps)
    solution = odeint(model, initial_conditions, t, args=(parameter_set,))
    traj = torch.zeros((num_trajectories, num_timesteps))
    for i in range(num_trajectories):
        for timestep in range(num_timesteps):
            traj[i, timestep] = solution[timestep][i]
    true_list = []
    for tra in traj: #Erase this loop if we want 6x1000
        for timepoint in tra:
            true_list.append(timepoint)
    true_tensor = torch.Tensor(true_list)
    return true_tensor

true_data = simulator(true_params)
true_data

tensor([  0.0000,   2.3466,   4.4394,  ..., 746.8439, 759.2465, 770.9217])

In [3]:
true_data.shape

torch.Size([6000])

BoxUniform(Uniform(low: torch.Size([2]), high: torch.Size([2])), 1)

In [15]:
batch_size = 2
prior.sample((batch_size,))

tensor([[162.0011, 109.5707],
        [ 35.1546,   7.2331]])

In [16]:
simulator(prior.sample((batch_size,)))

/project/home23/sg2023/Desktop/SBI/.conda/lib/python3.11/site-packages/scipy/integrate/_odepack_py.py:242: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  output = _odepack.odeint(func, y0, t, args, Dfun, col_deriv, ml, mu,


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6,) + inhomogeneous part.

In [14]:
# Try function's workings
batch_size = 1
# The simulator must return a matching batch dimension and data.
simulator(prior.sample((batch_size,))).shape

ValueError: not enough values to unpack (expected 2, got 1)

# Change simulator?

In [27]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer

# Define repressilator model
def model(variables, t, params):

    m1, p1, m2, p2, m3, p3 = variables
    k1, k2 = params #only 2 ks are parameters to infer
    k3 = 246.96291990024542
    a1 = a2 = a3 = 24.78485282457379
    g1 = g2 = g3 = 0.024884149937163258
    n1 = n2 = n3 = 5
    b1 = b2 = b3 = 33.82307682700831
    dm1 = dm2 = dm3 = 1.143402097500176
    dp1 = dp2 = dp3 = 0.7833664565550977

    dm1dt = -dm1 * m1 + (a1 / (1 + ((1/k1) * p2) ** n1)) + g1
    dp1dt = (b1 * m1) - (dp1 * p1)
    dm2dt = -dm2 * m2 + (a2 / (1 + ((1/k2) * p3) ** n2)) + g2
    dp2dt = (b2 * m2) - (dp2 * p2)
    dm3dt = -dm3 * m3 + (a3 / (1 + ((1/k3) * p1) ** n3)) + g3
    dp3dt = (b3 * m3) - (dp3 * p3)
    
    return [dm1dt, dp1dt, dm2dt, dp2dt, dm3dt, dp3dt]

# Define the true parameters for k
true_params = np.array([
    246.96291990024542, 246.96291990024542,])

# Establish prior from 0-250
num_dim = 2
prior = utils.BoxUniform(low=10**-2 * torch.ones(num_dim), high=250 * torch.ones(num_dim))

In [38]:
full_tensor = torch.tensor([])
full_tensor_ext = torch.cat((full_tensor, torch.tensor([1,2])), dim = 0)
full_tensor_ext2 = torch.cat((full_tensor_ext, torch.tensor([1,2])), dim = 0)
full_tensor_ext2.shape

torch.Size([4])

In [39]:
import torch

# Existing tensor with batch size 3, each tensor with shape (2, 3)
existing_tensor = torch.tensor([[[1, 2, 3], [4, 5, 6]],
                                [[7, 8, 9], [10, 11, 12]],
                                [[13, 14, 15], [16, 17, 18]]])

# Data to append with batch size 1, each tensor with shape (2, 3)
new_data = torch.tensor([[[19, 20, 21], [22, 23, 24]]])

# Concatenate tensors along the batch dimension (dim=0)
appended_tensor = torch.cat((existing_tensor, new_data), dim=0)

print("Appended tensor:")
print(appended_tensor)


Appended tensor:
tensor([[[ 1,  2,  3],
         [ 4,  5,  6]],

        [[ 7,  8,  9],
         [10, 11, 12]],

        [[13, 14, 15],
         [16, 17, 18]],

        [[19, 20, 21],
         [22, 23, 24]]])


In [41]:
existing_tensor.shape

torch.Size([3, 2, 3])

In [47]:
full_tensor = torch.zeros((num_trajectories, num_timesteps)).unsqueeze(0)
full_tensor_one = torch.ones((num_trajectories, num_timesteps)).unsqueeze(0)
out = torch.cat((full_tensor, full_tensor_one), dim=0)
out.shape

torch.Size([2, 6, 1000])

In [48]:
out[1]

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [50]:
out.size(0)

2

In [5]:
# We define the time variable (0-100 in 1000 steps), the initial conditions, etc.
num_timesteps = 1000
num_trajectories = 6
initial_conditions = np.array([0, 1, 0, 3, 0, 2])
t = np.linspace(0, 100, num_timesteps)

def simulator(parameter_set):
    full_tensor = torch.zeros((num_trajectories * num_timesteps)).unsqueeze(0)
    if len(parameter_set) == 1:
        for params in parameter_set:
            solution = odeint(model, initial_conditions, t, args=(params,))
            traj = torch.zeros((num_trajectories, num_timesteps))
            for i in range(num_trajectories):
                for timestep in range(num_timesteps):
                    traj[i, timestep] = solution[timestep][i]
            true_list = []
            for tra in traj: #Erase this loop if we want 6x1000
                for timepoint in tra:
                    true_list.append(timepoint)
            true_tensor = torch.Tensor(true_list).unsqueeze(0)
            return true_tensor
    else:
        for params in parameter_set:
            solution = odeint(model, initial_conditions, t, args=(params,))
            traj = torch.zeros((num_trajectories, num_timesteps))
            for i in range(num_trajectories):
                for timestep in range(num_timesteps):
                    traj[i, timestep] = solution[timestep][i]
            true_list = []
            for tra in traj: #Erase this loop if we want 6x1000
                for timepoint in tra:
                    true_list.append(timepoint)
            true_tensor = torch.Tensor(true_list).unsqueeze(0)
            full_tensor = torch.cat((full_tensor, true_tensor), dim=0)
        n_sims = full_tensor.size(0)
        output_tensor = full_tensor[1:n_sims]
        return output_tensor


In [6]:
true_params = torch.tensor([
    246.96291990024542, 246.96291990024542,]).unsqueeze(0)
true_params.shape

torch.Size([1, 2])

In [7]:
true_data = simulator(true_params)
true_data

tensor([[  0.0000,   2.3466,   4.4394,  ..., 746.8439, 759.2466, 770.9218]])

In [8]:
batch_size = 3
batch_params = prior.sample((batch_size,))
batch_params.shape

torch.Size([3, 2])

In [9]:
simulator(prior.sample((batch_size,))).shape

torch.Size([3, 6000])

In [90]:
simulator(prior.sample((batch_size,)))

tensor([[  0.0000,   2.3459,   4.3501,  ..., 609.4203, 580.2196, 551.4930],
        [  0.0000,   2.3466,   4.4394,  ...,  74.3807,  69.3614,  64.6759],
        [  0.0000,   2.3466,   4.4394,  ..., 360.3610, 339.5109, 319.5849]])

# simulator NOW HANDLES multiple param sets!

In [10]:
batch_size = 2
output_shape = simulator(prior.sample((batch_size,))).shape
output_shape

torch.Size([2, 6000])

In [13]:
len(output_shape) > 1

True

In [12]:
output_shape[0] == batch_size

True

# Add an input dimension of 1! for input_size to RNN of 1

In [15]:
# We define the time variable (0-100 in 1000 steps), the initial conditions, etc.
num_timesteps = 1000
num_trajectories = 6
initial_conditions = np.array([0, 1, 0, 3, 0, 2])
t = np.linspace(0, 100, num_timesteps)

def simulator(parameter_set):
    full_tensor = torch.zeros((num_trajectories * num_timesteps)).unsqueeze(0)
    if len(parameter_set) == 1:
        for params in parameter_set:
            solution = odeint(model, initial_conditions, t, args=(params,))
            traj = torch.zeros((num_trajectories, num_timesteps))
            for i in range(num_trajectories):
                for timestep in range(num_timesteps):
                    traj[i, timestep] = solution[timestep][i]
            true_list = []
            for tra in traj: #Erase this loop if we want 6x1000
                for timepoint in tra:
                    true_list.append(timepoint)
            true_tensor = torch.Tensor(true_list).unsqueeze(0)
            out = true_tensor.unsqueeze(-1)
            return out
    else:
        for params in parameter_set:
            solution = odeint(model, initial_conditions, t, args=(params,))
            traj = torch.zeros((num_trajectories, num_timesteps))
            for i in range(num_trajectories):
                for timestep in range(num_timesteps):
                    traj[i, timestep] = solution[timestep][i]
            true_list = []
            for tra in traj: #Erase this loop if we want 6x1000
                for timepoint in tra:
                    true_list.append(timepoint)
            true_tensor = torch.Tensor(true_list).unsqueeze(0)
            full_tensor = torch.cat((full_tensor, true_tensor), dim=0)
        n_sims = full_tensor.size(0)
        output_tensor = full_tensor[1:n_sims]
        out = output_tensor.unsqueeze(-1)
        return out


In [16]:
batch_size = 2
output_shape = simulator(prior.sample((batch_size,))).shape
output_shape

torch.Size([2, 6000, 1])

In [17]:
len(output_shape) > 1

True

In [18]:
output_shape[0] == batch_size

True

# BEWARE: this code is very unoptimised, just proof of concept